In [2]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
@ops.RegisterGradient("DubIdent")
def dubIdent(op, grad):
    if (grad.shape != op.inputs[0].shape):
        ident_out0 = tf.ones(tf.shape(op.inputs[0]))
    else:
        ident_out0 = grad
        
    if (grad.shape != op.inputs[1].shape):
        ident_out1 = tf.ones(tf.shape(op.inputs[1]))
    else:
        ident_out1 = grad
    return [ident_out0, ident_out1]

@ops.RegisterGradient("Bin_ST")
def bin_ST(op, grad):
    in_vals = op.inputs[0]
    grad_func = tf.cast(tf.less_equal(tf.abs(in_vals), 1), tf.float32)
    grad_out = tf.multiply(grad_func, grad)
    return [grad_out]

def Multibit(x, num_bits):    
    g = tf.get_default_graph()
    with ops.name_scope("Multibit") as name:
        with g.gradient_override_map({"Fill": "Identity", "Pow": "DubIdent", "Sum": "Identity", "Ceil": "Identity", "RealDiv": "DubIdent", "Max": "DubIdent", "Add": "DubIdent", "Sub": "DubIdent", "Mul": "DubIdent"}):
            norm = tf.clip_by_value(x, -1.0, 1.0)
            norm = tf.add(norm, 1)
            norm = tf.multiply(norm, tf.pow(2.0, num_bits-1))            
            #rounded = tf.subtract((tf.ceil(norm + 1e-8)), 1)
            rounded = tf.floor(norm)
            #rounded = tf.divide(rounded, tf.reduce_max(rounded))
            rounded = tf.divide(rounded, (tf.pow(2.0, num_bits) - 1))
            rounded = tf.subtract(rounded, .5)
            rounded = tf.multiply(rounded, 2.0)
            return rounded
    
def BuildMultibit(x, num_bits):
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": "Bin_ST"}):
        return tf.py_func(Multibit, [x, num_bits], tf.float32, stateful=True, name="Multibit")

In [13]:
#x = tf.constant([-1,8,-.3, .001, -.2])
x = tf.constant([-1, -.9, -.8, -.7, -.6, -.5, -.4, -.3, -.2, -.1, 0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1, 2], dtype=np.float32)
#test = tf.nn.l2_normalize(x, dim=0)
round_vals = tf.linspace(-1.0, 1.0, tf.pow(2, 3))
print(round_vals)
#test = tf.reduce_sum(x)
sess = tf.Session()
test_vals = sess.run(round_vals)
#sess.run(x)
test_vals

Tensor("LinSpace_4:0", shape=(?,), dtype=float32)


array([-1.        , -0.71428573, -0.4285714 , -0.14285707,  0.14285719,
        0.42857146,  0.71428585,  1.        ], dtype=float32)

In [14]:
test_op = Multibit(x, 2)
print(test_op)
#sess.run(test_op)
sess.run(test_op)

Tensor("Multibit_5/Mul_1:0", shape=(22,), dtype=float32)


array([-1.        , -1.        , -1.        , -1.        , -1.        ,
       -0.33333331, -0.33333331, -0.33333331, -0.33333331, -0.33333331,
        0.33333337,  0.33333337,  0.33333337,  0.33333337,  0.33333337,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.66666675,  1.66666675], dtype=float32)

In [5]:
out = tf.gradients([test_op], [x])
sess.run(out)

TypeError: Fetch argument None has invalid type <type 'NoneType'>

In [ ]:
sess.run(x)

In [ ]:
g = tf.get_default_graph()
g.get_operations()